<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Timing,-Profiling,-and-Pickling" data-toc-modified-id="Timing,-Profiling,-and-Pickling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Timing, Profiling, and Pickling</a></span><ul class="toc-item"><li><span><a href="#Time-Profiling" data-toc-modified-id="Time-Profiling-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Time Profiling</a></span><ul class="toc-item"><li><span><a href="#%time" data-toc-modified-id="%time-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>%time</a></span></li><li><span><a href="#%timeit" data-toc-modified-id="%timeit-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>%timeit</a></span></li><li><span><a href="#%prun" data-toc-modified-id="%prun-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>%prun</a></span></li></ul></li><li><span><a href="#Pickling" data-toc-modified-id="Pickling-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Pickling</a></span></li><li><span><a href="#Function-Caching" data-toc-modified-id="Function-Caching-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Function Caching</a></span><ul class="toc-item"><li><span><a href="#Twenty-million-times-faster" data-toc-modified-id="Twenty-million-times-faster-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Twenty-million times faster</a></span></li></ul></li></ul></li></ul></div>

# Timing, Profiling, and Pickling

Essential tools for profiling and optimizing code and creating repeatable environments. Plus, it's just downright fun!

A more in-depth guide with additional profiling tools can be found <a href="https://pynash.org/2013/03/06/timing-and-profiling/">here</a>.

## Time Profiling
Using Jupyter Notebook (IPython) we have several very convenient notebook magics for time profiling including `%time`, `%timeit`, and `%prun`.

In [4]:
# to see the docstring for each of these magics, run the following magic inquiries:
%time?
%timeit?
%prun?

### %time

Tells how long it takes to run a script

In [10]:
%time {1 for num in range(10000000)}

Wall time: 727 ms


{1}

### %timeit

Performs multiple runs of a script and returns the average time & std.dev.

In [11]:
%timeit {1 for num in range(10000000)}

627 ms ± 56.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# the -n flag lets you choose how many times to run the script
%timeit -n 100 {1 for num in range(10000000)}

663 ms ± 6.63 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### %prun

Tells how long it takes each function in a script to run.

In the tests below, we can see very clearly that the most efficient scripts have the fewest function calls. This tells us that the majority of our overhead comes from function calls. We know how to begin optimizing.

In [135]:
%prun

def fac0(n): 
    if n > 0:
        prod = 1
        for i in range(1, n+1): prod*=i
        return prod
    elif n==0: return 1
    else: raise ValueError("By definition operand of factorial must be an integer >= 0")

            
def errr(): raise ValueError("By definition operand of factorial must be >= 0")
fac1 = lambda n: 1 if n==0 else n*fac1(n-1) if n>0 else errr()


from itertools import *
mult = lambda x, y: x*y
fac2 = lambda x: [i for i in accumulate(range(1, x+1), mult)][-1]

import numpy as np
import numexpr as ne
ne.set_vml_accuracy_mode('high')
def fac3(n):
    if n > 0:
        n = np.asarray(range(1, n+1))
        return ne.evaluate("prod(n, 0)")
    elif n==0:
        return 1
    else: raise ValueError("By definition operand of factorial must be an integer >= 0")

In [173]:
%prun fac0(16)
%timeit -n 100 fac0(16)

 2.02 µs ± 16.7 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [145]:
%prun fac1(16)
%timeit -n 100 fac1(16)

 2.04 µs ± 7.16 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [146]:
%prun fac2(16)
%timeit -n 100 fac2(16)

 5.15 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [141]:
%prun fac3(16)
%timeit -n 100 fac3(16)

 31.7 µs ± 9.98 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now that we know the majority of our overhead comes from function calls, lets see if we can optimize any of these.

`fac0()` is about as optimized as it is going to get.

`fac1()` looks promising, but there is no way to avoid recursion, and therefor no way to eliminate the function calls that it uses without completely redesigning it.

`fac2()` has genuine potential. It relies on the `itertools` package. It creates an array and returns the last element of the array. The `accumulate()` statement repeatedly calls our `mult` function. Lets try using `reduce()` from `functools` as well as `operator.mul`.

`fac3()` has the most room for improvement, but it is so much slower than the others that we might as well not touch it for the time being. It relies on NumExpr which is meant for something entirely different (see <a href="https://github.com/pydata/numexpr/issues/301#issuecomment-388097698">here</a>) and cannot provide sufficient accuracy for factorial greater than 16.

In [174]:
# original function
from itertools import *
mult = lambda x, y: x*y
fac2_old = lambda x: [i for i in accumulate(range(1, x+1), mult)][-1]

# new function
from functools import reduce
from operator import mul
fac2_new = lambda n: 1 if n == 0 else reduce(mul, range(1,n)) if n>0 else errr()

In [175]:
%prun fac2_old(16)
%timeit -n 100 fac2_old(16)

 3.42 µs ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [176]:
%prun fac2_new(16)
%timeit -n 100 fac2_new(16)

 1.02 µs ± 8.37 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


Awesome! We reduced the number of function calls by 75% and reduced the run-time by approx 75%. This is now the fastest version of our function.

Can we do even better by optimizing `fac0()` in the same way, since `fac0()` does not rely on the `errr()` function to return a ValueError?

In [180]:
# old function
def fac0_old(n): 
    if n > 0:
        prod = 1
        for i in range(1, n+1): prod*=i
        return prod
    elif n==0: return 1
    else: raise ValueError("By definition operand of factorial must be an integer >= 0")
        
# new function
from functools import reduce
from operator import mul

def fac0_new(n): 
    if n > 0: return reduce(mul, range(1,n))
    elif n==0: return 1
    else: raise ValueError("By definition operand of factorial must be an integer >= 0")

In [182]:
%prun fac0_old(16)
%timeit -n 100 fac0_old(16)

 2.94 µs ± 877 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [183]:
%prun fac0_new(16)
%timeit -n 100 fac0_new(16)

 1.04 µs ± 95.4 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


Up front, it looks like `fac0_new()` and `fac2_new()` perform exactly the same, but `fac0_new()` doesn't require us to define `errr()`, so it is going to be slightly more efficient if not less compact.

## Pickling

*Pickling* is "the process whereby a Python object hierarchy is convertd into a bytestream, and "unpickling" is the inverse operation. This process is alternatively known as "serializion", "marshalling", or "flattening".

All pertinent information regarding `pickle` can be found <a href="https://docs.python.org/3/library/pickle.html">here</a>.

Pickling is a critical component in nearly all of the packages and tools that we'll be exploring.

## Function Caching

Function caching allows us to cache the return values of a function that is repeatedly called. It saves time when a function is repeatedly called with the same arguments.

In Python 3.2+ there is an `lru_cache` decorator which allows us to quickly perform the caching.

### Twenty-million times faster

In [5]:
# WITHOUT CACHING
def fib0(n):
    if n<2: return n
    return fib0(n-1) + fib0(n-2)

def fib1(n):
    return fib1(n-1) + fib1(n-2) if n>=2 else n

In [18]:
%timeit fib0(34)
%timeit fib1(34)

1.96 s ± 75.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.02 s ± 84.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
# with caching
from functools import lru_cache

@lru_cache(maxsize=32) # store the last 32 return values at most
def fib0(n):
    if n<2: return n
    return fib0(n-1) + fib0(n-2)

@lru_cache(maxsize=32) # store the last 32 return values at most
def fib1(n):
    return fib1(n-1) + fib1(n-2) if n>=2 else n

In [21]:
%timeit fib0(34)
%timeit fib1(34)

105 ns ± 9.89 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
103 ns ± 8.44 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


With caching, this simple Fibonacci script is liverally 20000000x faster (yes, that's **twenty-million** times faster).